In [0]:
#@title Authored by Michael Evans
# Senior Conservation Data Scientist
# Defenders of Wildlife. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
!pip install earthengine-api

In [0]:
import ee
ee.Authenticate()
ee.Initialize()
# Test the earthengine command by getting help on upload.
!earthengine upload image -h

In [0]:
from google.colab import auth, drive

auth.authenticate_user()

In [3]:
#Mount GDrive to notebook
ROOT = ('/content/drive')
drive.mount(ROOT)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
# make sure the python kernel can access our modules in GDrive
import sys
sys.path.append('/content/drive/My Drive/repos/ACD_methods/EEcode/Python')
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/My Drive/repos/ACD_methods/EEcode/Python']

In [0]:
# import change detection modules
import clouds, terrain, stats, dictionaries, iw

In [6]:
import folium
print(folium.__version__)

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

0.8.3


In [0]:
# define study variabes
S2 = ee.ImageCollection("COPERNICUS/S2")
SR = ee.ImageCollection("COPERNICUS/S2_SR")
CDL = ee.Image("USDA/NASS/CDL/2019")
DEM = ee.Image("USGS/SRTMGL1_003")

lpcRange = ee.FeatureCollection("users/defendersofwildlifeGIS/LPC/sgpchat_lpc_habitat_regions")
beforeStart = '2016-05-01'
beforeEnd = '2016-07-01'
afterStart = '2020-04-01'
afterEnd = '2020-05-26'
rgbn = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
habitat = dictionaries.scrub

In [0]:
# we will do one portion of LPC range at a time
lpcRange = lpcRange.map(lambda x: x.set('area', x.area(10)))
aoi = lpcRange.limit(1, 'area', True)
centroid = aoi.geometry().centroid()
coordinates = centroid.coordinates().getInfo()
coordinates.reverse()

In [0]:
# confirm we have selected the aoi we want
aoiImg = ee.Image().paint(aoi, color = 'red')
map = folium.Map(location = coordinates, zoom_start = 8)
map.add_ee_layer(aoiImg, vis_params = {}, name = 'aoi')
map

In [46]:
# create before and after images
before = S2.filterBounds(aoi).filterDate(beforeStart, beforeEnd).map(clouds.maskTOA)
after = SR.filterBounds(aoi).filterDate(afterStart, afterEnd).map(clouds.maskSR)
masked = before.select(rgbn).merge(after.select(rgbn))
corrected = terrain.c_correct(masked, rgbn, aoi, DEM)
print(corrected.size().getInfo())

curr = corrected.filterDate(afterStart, afterEnd);
prev = corrected.filterDate(beforeStart, beforeEnd);

image1 = prev.median().clip(aoi);
image2 = curr.median().clip(aoi);
print(image1.bandNames().getInfo())

Running c_correct algorithm
143
['illumination', 'B2', 'B3', 'B4', 'B8', 'B11', 'B12']


In [50]:
map = folium.Map(location = coordinates, zoom_start = 8)
map.add_ee_layer(image2, {'bands':['B4', 'B3', 'B2'], 'min':250, 'max':2500}, 'after')
map.add_child(folium.LayerControl())
map

In [0]:
# run the iw algorithm to detect changes
iwout = iw.runIW(prev, curr, aoi, 500, 12, 'yes').clip(aoi);
task = ee.batch.Export.image.toAsset(
    iwout.select(['cv_z', 'nbr_z', 'ndsi_z', 'ndvi_z', 'ndwi_z', 'rcvmax_z']),
    description = 'LPC_IW_update1',
    assetId = 'users/defendersofwildlifeGIS/LPC/LPC_IW_update1',
    region = aoi.geometry(),
    scale = 10,
    maxPixels = 1e13)
task.start()

In [14]:
# perform LDA on the output
iwout = ee.Image('users/defendersofwildlifeGIS/LPC/LPC_IW_update1')
scored = stats.ldaScore(iwout, ['cv_z', 'nbr_z', 'ndsi_z', 'ndvi_z', 'ndwi_z', 'rcvmax_z'], habitat).clip(aoi)

# Select pixels above pre-defined change thresholds, and perform erosion + dilation
change = scored.gte(habitat.get('lda')).focal_min(1, 'square', 'pixels').focal_max(1, 'square', 'pixels')

change = change.updateMask(change)

map = folium.Map(location = coordinates)
map.add_ee_layer(change, {'palette':'purple', 'min':0, 'max':1}, 'change')

EEException: ignored